In [136]:
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [137]:
#Ver en que carpeta estas
!pwd

/content


In [138]:
#Path de la carpeta de Trabajo
PATH="/content/DATA"

In [139]:
def crear_carpeta(PATH=PATH):
    if not os.path.isdir(PATH):#V o F si existe el archivo en el directorio
        os.makedirs(PATH)#si no existe lo creo
crear_carpeta()#llamo a la funcion

In [140]:
path = Path(PATH)
os.listdir(path)

['train_split.csv', 'test_split.csv']

In [141]:
train = pd.read_csv(f'{PATH}/train_split.csv')
train.head()

,P80_Planta,P80_Mina,BWI,FP,Throughput
0,4.20,2.9,10.50,0.16,Bueno
1,6.02,3.3,35.25,1.56,Normal
2,5.99,2.8,36.00,1.76,Deficiente
3,4.72,3.1,11.25,0.06,Bueno
4,6.71,3.1,40.50,2.06,Deficiente


In [142]:
test = pd.read_csv(f'{PATH}/test_split.csv')
test.head()

,P80_Planta,P80_Mina,BWI,FP,Throughput
0,4.11,3.0,9.75,0.16,Bueno
1,6.50,3.0,37.50,1.66,Normal
2,7.04,2.9,47.25,1.76,Deficiente
3,6.57,3.2,44.25,2.26,Deficiente
4,4.16,3.2,9.75,0.16,Bueno


In [143]:
import numpy as np
X_cat=train[["Throughput"]]
X_cat.shape

(120, 1)

In [144]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
X_cat = onehotencoder.fit_transform(X_cat).toarray()#El to array es xq por defecto te lo da en un sparce
onehotencoder.categories_

[array(['Bueno', 'Deficiente', 'Normal'], dtype=object)]

In [145]:
 X_cat = pd.DataFrame(X_cat)
 X_cat

,0,1,2
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0
4,0.0,1.0,0.0
...,...,...,...
115,1.0,0.0,0.0
116,1.0,0.0,0.0
117,1.0,0.0,0.0
118,0.0,1.0,0.0


In [146]:
X_numerical = train[['P80_Mina', 'BWI', 'FP']]
X_numerical

,P80_Mina,BWI,FP
0,2.9,10.50,0.16
1,3.3,35.25,1.56
2,2.8,36.00,1.76
3,3.1,11.25,0.06
4,3.1,40.50,2.06
...,...,...,...
115,3.1,11.25,0.06
116,3.2,9.75,0.16
117,4.2,10.50,0.16
118,3.0,44.25,2.06


In [147]:
X_all = pd.concat([X_cat, X_numerical], axis = 1)
X_all

,0,1,2,P80_Mina,BWI,FP
0,1.0,0.0,0.0,2.9,10.50,0.16
1,0.0,0.0,1.0,3.3,35.25,1.56
2,0.0,1.0,0.0,2.8,36.00,1.76
3,1.0,0.0,0.0,3.1,11.25,0.06
4,0.0,1.0,0.0,3.1,40.50,2.06
...,...,...,...,...,...,...
115,1.0,0.0,0.0,3.1,11.25,0.06
116,1.0,0.0,0.0,3.2,9.75,0.16
117,1.0,0.0,0.0,4.2,10.50,0.16
118,0.0,1.0,0.0,3.0,44.25,2.06


In [148]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_all = scaler.fit_transform(X_all)

In [149]:
X_all[:5]

array([[1.        , 0.        , 0.        , 0.375     , 0.05172414,
        0.04166667],
       [0.        , 0.        , 1.        , 0.54166667, 0.62068966,
        0.625     ],
       [0.        , 1.        , 0.        , 0.33333333, 0.63793103,
        0.70833333],
       [1.        , 0.        , 0.        , 0.45833333, 0.06896552,
        0.        ],
       [0.        , 1.        , 0.        , 0.45833333, 0.74137931,
        0.83333333]])

In [150]:
y = train['P80_Planta']
y[:5]

0    4.20
1    6.02
2    5.99
3    4.72
4    6.71
Name: P80_Planta, dtype: float64

In [151]:
X_all.shape,y.shape

((120, 6), (120,))

In [152]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_all,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [153]:
print("Predictions:", lin_reg.predict(X_all[:5]))

Predictions: [4.51041445 6.25076908 5.82166667 4.71351387 6.32986774]


In [154]:
print("Labels:", y[:5])

Labels: 0    4.20
1    6.02
2    5.99
3    4.72
4    6.71
Name: P80_Planta, dtype: float64


In [155]:
from sklearn.metrics import mean_squared_error
voladura_predictions = lin_reg.predict(X_all)
lin_mse = mean_squared_error(y, voladura_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

0.29955155640681497

In [156]:
from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y, voladura_predictions)
lin_mae

0.25150436209710914

In [157]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X_all,y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [158]:
voladura_predictions = tree_reg.predict(X_all)
tree_mse = mean_squared_error(y, voladura_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0029814239699997224

In [159]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, X_all,y,
 scoring="neg_mean_squared_error", cv=3)
tree_rmse_scores = np.sqrt(-scores)

In [160]:
def display_scores(scores):
  print("Scores:", scores)
  print("Mean:", scores.mean())
  print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

Scores: [0.38948363 0.39485124 0.4908895 ]
Mean: 0.4250747896986417
Standard deviation: 0.046589589280463105


In [161]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(X_all,y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [162]:
voladura_predictions = forest_reg.predict(X_all)
forest_mse = mean_squared_error(y, voladura_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

0.11200357094859155

In [163]:
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_reg, X_all,y,
                                scoring="neg_mean_squared_error", cv=3)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [0.31252238 0.32340618 0.36464314]
Mean: 0.3335238983976366
Standard deviation: 0.022448750865493868


In [164]:
scores = cross_val_score(lin_reg, X_all,y, scoring="neg_mean_squared_error", cv=10)
pd.Series(np.sqrt(-scores)).describe()

count    10.000000
mean      0.321944
std       0.057778
min       0.176198
25%       0.305397
50%       0.342345
75%       0.350426
max       0.386046
dtype: float64

In [165]:
from sklearn.svm import SVR
svm_reg = SVR(kernel="linear")
svm_reg.fit(X_all,y)
voladura_predictions = svm_reg.predict(X_all)
svm_mse = mean_squared_error(y, voladura_predictions)
svm_rmse = np.sqrt(svm_mse)
svm_rmse

0.3507647264251666

In [171]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2,4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X_all,y)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs

In [172]:
grid_search.best_params_

{'max_features': 2, 'n_estimators': 30}

In [173]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=2, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=30, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [169]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

0.3548687788546089 {'max_features': 2, 'n_estimators': 3}
0.34972884607618204 {'max_features': 2, 'n_estimators': 10}
0.33497636760817484 {'max_features': 2, 'n_estimators': 30}
0.3789627906639633 {'max_features': 4, 'n_estimators': 3}
0.34853742989388825 {'max_features': 4, 'n_estimators': 10}
0.3404254379606486 {'max_features': 4, 'n_estimators': 30}
0.35523781250727554 {'max_features': 6, 'n_estimators': 3}
0.33386428378368743 {'max_features': 6, 'n_estimators': 10}
0.3270844033502787 {'max_features': 6, 'n_estimators': 30}
nan {'max_features': 8, 'n_estimators': 3}
nan {'max_features': 8, 'n_estimators': 10}
nan {'max_features': 8, 'n_estimators': 30}
0.3554806055263069 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
0.35129199762105695 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
0.3697643699714881 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
0.3518396349243555 {'bootstrap': False, 'max_features': 3, 'n_estimators': 10}
0.3909973924533894 {'

In [170]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_n_estimators,param_bootstrap,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.004980,0.000615,0.000883,0.000115,2,3,NaN,"{'max_features': 2, 'n_estimators': 3}",-0.084252,-0.172580,-0.112051,-0.113746,-0.147030,-0.125932,0.030658,9,-0.037357,-0.019238,-0.032739,-0.029722,-3.609769e-02,-0.031031,0.006471
1,0.013976,0.001632,0.001566,0.000283,2,10,NaN,"{'max_features': 2, 'n_estimators': 10}",-0.081194,-0.181674,-0.097088,-0.144788,-0.106807,-0.122310,0.036322,6,-0.020734,-0.014240,-0.018678,-0.016841,-1.798534e-02,-0.017696,0.002143
2,0.041160,0.002921,0.002601,0.000061,2,30,NaN,"{'max_features': 2, 'n_estimators': 30}",-0.079446,-0.167756,-0.069492,-0.149382,-0.094969,-0.112209,0.039148,3,-0.019489,-0.014043,-0.017843,-0.015594,-1.462953e-02,-0.016320,0.002046
3,0.004584,0.000038,0.000834,0.000024,4,3,NaN,"{'max_features': 4, 'n_estimators': 3}",-0.095601,-0.224398,-0.088560,-0.195994,-0.113512,-0.143613,0.055699,14,-0.027849,-0.019771,-0.025936,-0.027547,-2.252187e-02,-0.024725,0.003117
4,0.013401,0.000479,0.001417,0.000091,4,10,NaN,"{'max_features': 4, 'n_estimators': 10}",-0.077034,-0.180337,-0.074752,-0.168337,-0.106931,-0.121478,0.044789,5,-0.018297,-0.012753,-0.016213,-0.017975,-1.719675e-02,-0.016487,0.002001
5,0.040126,0.003041,0.002717,0.000312,4,30,NaN,"{'max_features': 4, 'n_estimators': 30}",-0.066184,-0.173337,-0.075475,-0.158121,-0.106330,-0.115889,0.043079,4,-0.017901,-0.012906,-0.017955,-0.014838,-1.425453e-02,-0.015571,0.002024
6,0.004786,0.000143,0.000877,0.000019,6,3,NaN,"{'max_features': 6, 'n_estimators': 3}",-0.094410,-0.184095,-0.083230,-0.143045,-0.126190,-0.126194,0.036035,10,-0.036654,-0.017250,-0.025126,-0.023625,-2.208877e-02,-0.024949,0.006423
7,0.014314,0.000493,0.001457,0.000046,6,10,NaN,"{'max_features': 6, 'n_estimators': 10}",-0.076272,-0.172720,-0.077898,-0.135635,-0.094802,-0.111465,0.037361,2,-0.019257,-0.012282,-0.016955,-0.017777,-1.688209e-02,-0.016630,0.002336
8,0.038238,0.000673,0.002568,0.000026,6,30,NaN,"{'max_features': 6, 'n_estimators': 30}",-0.065568,-0.168738,-0.060351,-0.137908,-0.102356,-0.106984,0.041668,1,-0.017720,-0.012425,-0.016783,-0.014288,-1.352776e-02,-0.014949,0.001993
9,0.002599,0.000087,0.000000,0.000000,8,3,NaN,"{'max_features': 8, 'n_estimators': 3}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
